In [ ]:
from tensorflow.keras.datasets import boston_housing
(train_data, train_targets), (test_data, test_targets) = (
    boston_housing.load_data())

We don't have a very big dataset here... it's just 506 rows, total, 13 features.

In [ ]:
print("Train data dimensions:",train_data.shape)
print("Test data dimensions:",test_data.shape)
print(train_targets) # these are home prices in $1,000s of dollars (in the 70s that was a lot of money)

Let's whiten the data...

In [ ]:
# We can white the data like this; or we can use the np.mean(), np.sub(), and np.divide() functions.
mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std
test_data -= mean
test_data /= std

And, let's define our simple network. We are going to define a simple function that re-specifies a model and compiles it. We're doing this because we will use this inside a nested loop that implements k-fold cross-validation. This is something you've probably already learned in other classes, but when your sample is small, it's generally not a good idea to rely on single random holdout for model evaluation. 

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

def build_model():
    model = keras.Sequential([
        layers.Dropout(0.1),
        layers.BatchNormalization(), # This whites the inputs to the next layer (de-mean, divide by SD, which can help with training)
        layers.Dense(8, activation="relu"),
        layers.Dropout(0.1),
        layers.BatchNormalization(),
        layers.Dense(4, activation="relu"),
        layers.Dense(1)
    ])
    model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])
    return model

The example code from the book breaks up the training sample into k folds, and cycles over each, leveraging the *other* k-1 folds for training, and using the focal fold evaluation. We store the average validation error metric across the k runs. We can repeat this process using different numbers of epochs, batch sizes, etc. Eventually, when we settle on a model that looks good, we do a final evaluation on the test data.  

In [ ]:
k = 4 
num_val_samples = len(train_data) // k # floor division (i.e., round down to nearest integer.)
num_epochs = 500
all_mae_histories = []  

print("In total, we have",len(train_data),"training observations.")
print("With a k of",k,"we have",num_val_samples,"observations per fold.\n")

for i in range(k): # the folds are going to be indexed 0 through 3 if k = 4
    print("Processing fold #:",i)
    # if I slice past the end of the array, it just gives me what it can find! No errors.
    # This is important here, because the last fold won't produce an error, despite our slice going well beyond the end of the array.
    print("Validation data includes observations",i*num_val_samples,"through",(i+1)*num_val_samples-1) # minus 1 because a slice is up to and not including the second index.
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples] 
    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]
    print("Training data includes observations 0 through",i*num_val_samples-1,"joined with observations",(i+1)*num_val_samples,"through the final observation.\n")
    partial_train_data = np.concatenate(
        [train_data[:i * num_val_samples],
         train_data[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [train_targets[:i * num_val_samples],
         train_targets[(i + 1) * num_val_samples:]],
        axis=0)
    model = build_model()
    history = model.fit(partial_train_data, partial_train_targets,
                        validation_data=(val_data, val_targets),
                        epochs=num_epochs, batch_size=16, verbose=0)
    mae_history = history.history['val_mae']
    all_mae_histories.append(mae_history)

Now we can calculate the average MAE in each epoch, across the 4 iterations. That is, for epoch 1, take the average MAE over the 4 iterations, then do it again for epoch 2, and so on. We will end up with 500 averages.  

In [ ]:
average_mae_history = [np.mean([x[i] for x in all_mae_histories]) for i in range(num_epochs)]
len(average_mae_history)

And now we plot them.

In [ ]:
import matplotlib.pyplot as plt

plt.plot(average_mae_history[10:])
plt.xlabel("Epochs")
plt.ylabel("Validation MAE")
plt.show()

Now we fit a final model and evaluate its performance on the test data. Being off by ~$2,500 in 70's dollars is a lot actually. Our model isn't that great. 

In [ ]:
model = build_model()
model.fit(train_data, train_targets,
          epochs=150, batch_size=16, verbose=0)
test_mse_score, test_mae_score = model.evaluate(test_data, test_targets)

And of course, this is how we obtain predictions from the model. 

In [ ]:
predictions = model.predict(test_data)
print(predictions)